# Uporaba SQLite v Pythonu

Za priklop na bazo SQLite v Pythonu uporabimo vgrajeni modul `sqlite3`.

In [ ]:
import sqlite3

# testna baza
baza = "primer.db"

Ustvarimo tabelo in jo napolnimo.

In [ ]:
with sqlite3.connect(baza) as con:
    cur = con.cursor()   # "odzivnik" za pregledovanje poizvedbe
    cur.execute("DROP TABLE IF EXISTS Cars")
    cur.execute("CREATE TABLE Cars(Id INTEGER PRIMARY KEY, Name TEXT, Price INTEGER)")
    cur.execute("INSERT INTO Cars VALUES (1, 'Audi', 52642)")
    cur.execute("INSERT INTO Cars VALUES (2, 'Mercedes', 57127)")
    cur.execute("INSERT INTO Cars VALUES (3, 'Skoda', 9000)")
    cur.execute("INSERT INTO Cars VALUES (4, 'Volvo', 29000)")
    cur.execute("INSERT INTO Cars VALUES (5, 'Bentley', 350000)")
    cur.execute("INSERT INTO Cars VALUES (6, 'Citroen', 21000)")
    cur.execute("INSERT INTO Cars VALUES (7, 'Hummer', 41400)")
    cur.execute("INSERT INTO Cars VALUES (8, 'Volkswagen', 21600)")

Povezavo si lahko tudi shranimo in večkrat uporabimo.

In [ ]:
con = sqlite3.connect(baza)

Primer: Katere tabele so v bazi?

In [ ]:
with con:
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
    print(cur.fetchall())

Primer: Katere vrstice so v tabeli?

In [ ]:
with con:
    cur = con.cursor()
    cur.execute("SELECT * from Cars")
    print(cur.fetchall())

In [ ]:
print(cur.fetchone())

Ko kurzor enkrat vrne podatke, se "izčrpa".

In [ ]:
cur.fetchall()

Z `with` smo naredili transakcijo - ob uspešnem koncu se je potrdila, sicer bi se preklicala.

In [ ]:
with con:
    cur.execute("INSERT INTO Cars VALUES (9, 'Hyundai', 19200)")
    cur.execute("INSERT INTO Cars VALUES (9, 'Ferrari', 192000)")

Transakcije lahko potrjujemo in preklicujemo tudi sami.

In [ ]:
cur.execute("INSERT INTO Cars VALUES (10, 'Hyundai', 19200)")

In [ ]:
with sqlite3.connect(baza) as con2: # uporabimo novo povezavo
    cur2 = con2.cursor()
    cur2.execute("SELECT * from Cars")
    print(cur2.fetchall())

In [ ]:
con.rollback()

In [ ]:
con.commit()

Primer: Poizvedba z iteracijo po 'odzivniku'.

In [ ]:
with con:
    cur = con.cursor()
    cur.execute("SELECT * from Cars")
    prvi = cur.fetchone()
    print("Prvi:", prvi)
    for podatek in cur:
        print(podatek)
        if podatek[0] == 5:
            break
    print(cur.fetchall())

Primer interaktivne poizvedbe,  uporaba znaka `?`.

In [ ]:
with con:
    cur = con.cursor()
    kaj = input("Cena katerega vozila te zanima:")
    cur.execute("SELECT Name, Price FROM Cars WHERE Name LIKE ?", (kaj,))
    print(cur.fetchall())

Primer interaktivne poizvedbe, uporaba `:ključ`, preverjanje rezultata.

In [ ]:
with con:
    cur = con.cursor()
    kaj = input("Katero vozilo te zanima:").strip()
    cur.execute("SELECT Price FROM Cars WHERE Name LIKE :ime", {"ime": kaj})
    rez = cur.fetchone()
    if rez is None:
        print("Ni takega vozila ({0}).".format(kaj))
    else:
        print("Cena vozila {0} je {1}.".format(kaj, rez[0]))

Primer: večkratno vstavljanje.

In [ ]:
cars = [
    (17, 'Audi2', 52643),
    (11, 'MercedesX', 57642),
    (12, 'Škoda', 9000),
    (13, 'Volvek', 29000),
    (14, 'Bentley7', 350000),
    (15, 'HummerU', 41400),
    (16, 'VolkswagenPassat', 21600)
]
print("Vstavljamo: ", cars)
with con:
    cur = con.cursor()
    cur.executemany("INSERT INTO Cars VALUES (?, ?, ?)", cars)
    cur.execute("SELECT * from Cars")
    print("V tabeli imamo:")
    print(cur.fetchall())

Uporaba generatorja vrstic `sqlite3.Row`. Namesto $n$-terk za vrstice vračamo objekte.

In [ ]:
with con:
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    cur.execute("SELECT * from Cars")
    for vrstica in cur:
        print(vrstica["Name"])

Ko povezave več ne potrebujemo, jo zapremo.

In [ ]:
con.close()